In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("/Users/raghavraahul/Downloads/Fake.csv")
df2 = pd.read_csv("/Users/raghavraahul/Downloads/True.csv")

In [3]:
df1["Label"] = [1]*df1.shape[0]
df2['Label'] = [0]*df2.shape[0]
df = pd.concat([df1, df2], axis = 0)
#print(df.shape)
df.head()

,title,text,subject,date,Label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [4]:
df = df.drop(["subject", "date", "title"], axis = 1)
df.head()

,text,Label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [5]:
df.shape

(44898, 2)

In [6]:
df.Label.value_counts()

1    23481
0    21417
Name: Label, dtype: int64

In [7]:
import gensim.downloader as api
wv = api.load("word2vec-google-news-300") #specifying the dataset or type of word embeddings you want to download
#this model has been trained on Google News Articles

In [10]:
#get word vector of a word

#example: word vector of the word "great"
wv["great"]

array([ 7.17773438e-02,  2.08007812e-01, -2.84423828e-02,  1.78710938e-01,
        1.32812500e-01, -9.96093750e-02,  9.61914062e-02, -1.16699219e-01,
       -8.54492188e-03,  1.48437500e-01, -3.34472656e-02, -1.85546875e-01,
        4.10156250e-02, -8.98437500e-02,  2.17285156e-02,  6.93359375e-02,
        1.80664062e-01,  2.22656250e-01, -1.00585938e-01, -6.93359375e-02,
        1.04427338e-04,  1.60156250e-01,  4.07714844e-02,  7.37304688e-02,
        1.53320312e-01,  6.78710938e-02, -1.03027344e-01,  4.17480469e-02,
        4.27246094e-02, -1.10351562e-01, -6.68945312e-02,  4.19921875e-02,
        2.50000000e-01,  2.12890625e-01,  1.59179688e-01,  1.44653320e-02,
       -4.88281250e-02,  1.39770508e-02,  3.55529785e-03,  2.09960938e-01,
        1.52343750e-01, -7.32421875e-02,  2.16796875e-01, -5.76171875e-02,
       -2.84423828e-02, -3.60107422e-03,  1.52343750e-01, -2.63671875e-02,
        2.13623047e-02, -1.51367188e-01,  1.04003906e-01,  3.18359375e-01,
       -1.85546875e-01,  

In [11]:
df.head()

,text,Label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [12]:
#convert text to vector using gensim's word embedding 
#but before that some preprocessing, removing stop word, doing lemmatization

#spacy needed for preprocessing
import spacy
nlp = spacy.load("en_core_web_lg")

In [13]:
def preprocess_and_vectorize(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if not token.is_stop and not token.is_punct:
            filtered_tokens.append(token.lemma_)
    
    return filtered_tokens

In [15]:
preprocess_and_vectorize("Don't worry if you don't understand")

['worry', 'understand']

In [29]:
#to get sentence vector, get vector for each word in the sentence after preprocessing and average all the vectors
wv.get_mean_vector(['worry', 'understand'], pre_normalize=False)[:3]
#pre_nromalize set to True is better for machine learning. normalization is good practice

array([ 0.00976562, -0.00561523, -0.08905029], dtype=float32)

In [27]:
v1 = wv["worry"]
v2 = wv["understand"]

In [28]:
import numpy as np
np.mean([v1,v2], axis = 0)[:3]

array([ 0.00976562, -0.00561523, -0.08905029], dtype=float32)

In [30]:
wv.get_mean_vector(['worry', 'understand'])[:3]

array([ 0.00235079, -0.00284596, -0.03638233], dtype=float32)

In [32]:
#updated preprocess function returning mean vector
def preprocess_and_vectorize(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if not token.is_stop and not token.is_punct:
            filtered_tokens.append(token.lemma_)
    
    return wv.get_mean_vector(filtered_tokens)

In [33]:
preprocess_and_vectorize("Don't worry if you don't understand")

array([ 0.00235079, -0.00284596, -0.03638233,  0.00413919, -0.10635224,
        0.05758579,  0.13348952, -0.00689176,  0.05995331, -0.02875906,
       -0.02279907, -0.06939262, -0.01549996,  0.03538099, -0.0873947 ,
        0.10044542,  0.02490648,  0.11501945,  0.00442711, -0.07228819,
       -0.04447255,  0.02785169,  0.03338735,  0.02222995,  0.07053161,
        0.06723307,  0.03483712, -0.0873695 ,  0.04152397, -0.0969665 ,
       -0.00914938,  0.00555944, -0.0277799 , -0.00701522,  0.05281431,
       -0.01066206,  0.02466576,  0.02378148,  0.0279402 ,  0.05527755,
        0.01359304, -0.01062085,  0.06734448, -0.03492254, -0.08421434,
       -0.04324378, -0.03578918, -0.00817786, -0.02093195,  0.01856531,
       -0.07064191,  0.05023994, -0.03036207, -0.0412168 , -0.00684169,
        0.05189689, -0.04097777, -0.05789134,  0.044417  , -0.0470418 ,
       -0.01475445,  0.01373999, -0.00459672,  0.00419459,  0.04307397,
       -0.01500929, -0.08037488,  0.05358911, -0.0211829 , -0.00

In [34]:
df["vector"] = df["text"].apply(lambda x: preprocess_and_vectorize(x))

In [35]:
df.head()

,text,Label,vector
0,Donald Trump just couldn t wish all Americans ...,1,"[0.0077023148, 0.021570027, -0.002596029, 0.02..."
1,House Intelligence Committee Chairman Devin Nu...,1,"[-0.0009021067, 0.012329693, -0.0043598576, 0...."
2,"On Friday, it was revealed that former Milwauk...",1,"[0.009586705, 0.014328364, -0.004771368, 0.010..."
3,"On Christmas day, Donald Trump announced that ...",1,"[0.005762615, 0.011761372, -0.0010594468, 0.02..."
4,Pope Francis used his annual Christmas Day mes...,1,"[0.020733831, 0.01776746, 0.0067342864, 0.0293..."


In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.Label, test_size = 0.2, random_state = 2023)

In [37]:
import numpy as np

X_train_2d = np.stack(X_train) #this is done because by default X_train is a numpy array where each element 
#is also a numpy array, but it is not a native two dimensional array. stack operations makes it a 2D array
X_test_2d = np.stack(X_test)

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

clf = GradientBoostingClassifier()
clf.fit(X_train_2d, y_train)
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      4426
           1       0.97      0.96      0.96      4554

    accuracy                           0.96      8980
   macro avg       0.96      0.96      0.96      8980
weighted avg       0.96      0.96      0.96      8980



In [39]:
#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[4281,  145],
       [ 183, 4371]])